# Converter to Standard cdli-conll Data

In [3]:
import pandas as pd
import re

In [7]:
# import pyconll
# data = pyconll.load_from_file("./output/P000001.conll")

In [70]:
def read_conll(filepath):
    # read in the conll file
    data = pd.read_fwf(filepath)
    # extract useful information from the conll file
    useful_info = data.iloc[:,0].str.split('\t')
    # prepare a clean dataframe
    cleaned_data = pd.DataFrame({'ID': [], 'WORD': [], 'SEGM': [], 'POS': [], \
        'MORPH': [], 'HEAD': [], 'EDGE': [], 'MISC': []})
    # update the clean dataframe
    for i in range(cleaned_data.shape[1] - 1):
        cleaned_data.iloc[:, i] = [useful_info[row][i] for row in range(len(useful_info))]
    cleaned_data.iloc[:, -1] = [useful_info[row][-1] for row in range(len(useful_info))]

    return cleaned_data

In [77]:
df = read_conll("./royal_subcorpus_data/P216736.conll")
df

,ID,WORD,SEGM,POS,MORPH,HEAD,EDGE,MISC
0,1,da-da,da-da[3],PN,PN,11,ERG,_
1,2,ensi2,ensi2[ruler],N,N,1,appos,_
2,3,szuruppak{ki},szuruppak{ki}[1],SN,SN.GEN,2,GEN,_
3,4,ha-la-ad-da,ha-la-ad-da[1],PN,PN,1,appos,_
4,5,ensi2,ensi2[ruler],N,N,1,appos,_
5,6,szuruppak{ki},szuruppak{ki}[1],SN,SN.GEN,5,GEN,_
6,7,dumu-ni,dumu[child],N,N.3-SG-H-POSS.ERG,1,appos,_
7,8,ad-us2,ad-us2[plank],N,N.ABS,11,ABS,_
8,9,abul,abul[gate],N,N,11,LOC,_
9,10,{d}sud3-da-ke4,{d}sud3[1],DN,DN.GEN.L3-NH,9,GEN,_


In [78]:
# rename the WORD and MORPH column
df = df.rename(columns={"WORD": "FORM", "MORPH": "XPOSTAG"})
# drop the EDGE and POS columns
df = df.drop(columns = ['EDGE', 'POS'])

df

,ID,FORM,SEGM,XPOSTAG,HEAD,MISC
0,1,da-da,da-da[3],PN,11,_
1,2,ensi2,ensi2[ruler],N,1,_
2,3,szuruppak{ki},szuruppak{ki}[1],SN.GEN,2,_
3,4,ha-la-ad-da,ha-la-ad-da[1],PN,1,_
4,5,ensi2,ensi2[ruler],N,1,_
5,6,szuruppak{ki},szuruppak{ki}[1],SN.GEN,5,_
6,7,dumu-ni,dumu[child],N.3-SG-H-POSS.ERG,1,_
7,8,ad-us2,ad-us2[plank],N.ABS,11,_
8,9,abul,abul[gate],N,11,_
9,10,{d}sud3-da-ke4,{d}sud3[1],DN.GEN.L3-NH,9,_
